# Particle Swarm Optimization

Sources:
- http://www.swarmintelligence.org/tutorials.php
- https://github.com/tisimst/pyswarm/blob/master/pyswarm/pso.py

At iteration $k$, particle $i$ has posistion $x^i_k$ defined recursively as:

$$x^i_{k+1} = x^i_k + v^i_{k+1}$$

where $v$ is its velocity defined as follows:

$$ v^i_{k+1} = w_k \cdot v^i_{k} + c_1 r_1 (p^i_k - x^i_k) + c_2 r_2 (p^g_k - x^i_k)$$

Where:

- $p^i_k$ is the particle's best known position
- $p^g_k$ is the global (all particles) best known position
- $w_k$ is an inertia parameter
- $c_1$ is the cognitive parameter
- $c_2$ is the social parameter
- $r_1, r_2$ are random numbers in $(0, 1)$

In plain English, the velocity update is defined as:

new_velocity = (inertia * previous_velocity) + (move in the direction of the personal best) + (move in the direction of the global best)

In [2]:
import numpy as np
import random

In [134]:
# -- global parameters
c1 = 1
c2 = 2
w = .5

# -- Particle Class

class Particle:
    """A Python Class for a simple particle."""
    
    def __init__(self, upper, lower, ndim):
        self.position = np.array([random.uniform(lower, upper) for _ in range(ndim)])
        self.velocity = np.array([random.uniform(-1, 1) for _ in range(ndim)])
        self.personal_best_position = self.position
        self.personal_best_error = np.inf
    
    def __str__(self):
        #return("I am a particle with best position {},\ncurrent position {},\nand velocity {}.\n"
        #       .format(self.personal_best_position, self.position, self.velocity))
        return str(self.position)
    
    def current_error(self, function):
        return function(self.position)
    
    def update_velocity(self, global_best):
        r1 = random.random()
        r2 = random.random()
        self.velocity = w*self.velocity + c1*r1*(self.personal_best_position - self.position) + c2*r2*(global_best - self.position)
            
    def move(self):
        self.position = self.position + self.velocity
        # need to deal with when the particle goes out of bound...

In [162]:
# --- Solver Class
class PSO:
    
    """Solver instance for Particle Swarm Optimization."""
    
    def __init__(self, num_particles, function, n_iter, lower = -10, upper = 10, ndim = 3):
        self.particles = [Particle(lower, upper, ndim) for _ in range(num_particles)]
        self.global_best = np.array([])
        self.global_best_error = np.inf
        self.function = function
        self.n_iter = n_iter
        
    def update_best(self):
        for particle in self.particles:
            if particle.current_error(self.function) < self.global_best_error:
                self.global_best = particle.position
                self.global_best_error = particle.current_error(self.function)
                
    def move_particles(self):
        for particle in self.particles:
            particle.update_velocity(self.global_best)
            particle.move()
    
    def __str__(self):
        return """Current best position: {}
        With error: {}""".format(self.global_best, self.global_best_error)
    
    def go(self):
        
        for _ in range(self.n_iter):
            pso.update_best()
            pso.move_particles()
            if _ % 50 == 0:
                print(_)
                print(self.global_best_error)
                print("\n")
        print("Found minimum at {} with value {}.".format(self.global_best, self.global_best_error))

### Tests

In [163]:
def sumsquares(x):
    return(sum([i**2 for i in x]))

In [164]:
pso = PSO(100, sumsquares, n_iter = 100)
pso.go()

0
6.946184086267664


50
0.0004253457305378732


Found minimum at [0.00607099 0.00245675 0.00296391] with value 5.167731710142975e-05.


In [167]:
def shiftedsumsquares(x):
    return(sum([(i+2)**2 for i in x])) # minimum should be all -2

In [171]:
pso = PSO(20, shiftedsumsquares, n_iter = 100) # works well even with 20 swarms!
pso.go()

0
19.417415544303


50
0.0020972420741141033


Found minimum at [-2.00410702 -2.0122623  -2.01415294] with value 0.0003675372419485579.


## Parallelizing

In [ ]:
#...